In [2]:
# imports
import pandas as pd
import numpy as np
import json, time, random
import os.path
from urllib.request import urlopen
from pandas.io.json import json_normalize
from datetime import datetime, date

In [59]:
# config mappings
module_map = {
    "bs": ["balanceSheetHistory", "balanceSheetStatements"],
    "cf": ["cashflowStatementHistory", "cashflowStatements"],
    "is": ["incomeStatementHistory", "incomeStatementHistory"],
    "bs_q": ["balanceSheetHistoryQuarterly", "balanceSheetStatements"],
    "cf_q": ["cashflowStatementHistoryQuarterly", "cashflowStatements"],
    "is_q": ["incomeStatementHistoryQuarterly", "incomeStatementHistory"],
    
    "key_features": ["defaultKeyStatistics"],
    "profile":["assetProfile"],
    "officers":["assetProfile"]
}

JSON_URL = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}" \
    "?formatted=true&lang=en-US&region=US&modules={1}&corsDomain=finance.yahoo.com"
JSON_URL_QUOTE = "https://query1.finance.yahoo.com/v7/finance/quote?formatted=true&lang=en-US&region=US&symbols={0}&corsDomain=finance.yahoo.com"

conf_file = 'config.json'
config = load_config(conf_file)
tickers = config['symbols']

print("Universe: ", len(tickers), "companies")

Loading config.json
Universe:  197 companies


In [ ]:
# labels for operations and summary labels
cash_labels = ['cash', 'shortTermInvestments', 'longTermInvestments']
debt_labels = ['shortLongTermDebt', 'longTermDebt']
tev_labels = ['marketCap', 'netDebt', 'minorityInterest']

descriptive_cols = ["date", "periodEnded", "industry"]
enterprise_value_cols = [
    "postMarketPrice", "sharesOutstanding", "marketCap", "totalCash", "totalDebt", "netDebt",
    "minorityInterest", "enterpriseValue"]
intrinsic_value_cols = [
    "avgNoGrowthFCF", "projectedPeriod", "5yearGrowth", "perpGrowth",
    "discountRate", "multiple", "equityValue", "intrinsicValuePerShare",
    "marginOfSafety"]

basic_value_layout = []
basic_value_layout.extend(descriptive_cols)
basic_value_layout.extend(enterprise_value_cols)
basic_value_layout.extend(intrinsic_value_cols)

In [54]:
# utility functions
def save_config(config, fname):
    with open(fname, 'w') as file:
        data = json.dumps(config, indent=1)
        file.write(data)
        file.close()
        print('Saving', fname)

def load_config(fname):
    with open(fname, 'r') as file:
        data = file.read()
        file.close()
        print('Loading', fname)
        return json.loads(data)

def load_store_ds(module, tickers):
    flag, ds = check_for_symbol(module, None)
    for t in tickers:
        record = list(map(functions[module], [t]))[0]
        if t not in ds.index: ds = ds.append(record)
    ds.to_csv(DATA_PATH + module + '.csv')
    return ds

def if_col_value(df, col_name):
    return df[col_name] if (df.columns.contains(col_name)) else 0

def total_fin_subset(df, columns):
    """Sums a subset of columns"""
    fin_section = df[columns]
    return fin_section.sum(axis=1).sort_index(ascending=True)

def fin_append_ltm(last4A, last4Q):
    """Calculates LTM (sum of last four quarters) and returns appended DataFrame"""
    last4Q.sort_index(ascending=True, inplace=True)
    last4A.sort_index(ascending=True, inplace=True)
    if last4A.index[-1] != last4Q.index[-1]:
        last4A.loc[last4Q.index[-1]] = last4Q.sum()
    return last4A.sort_index()

def available_labels(df, labels):
    """Returns the labels contained in the DataFrame"""
    return [x for x in labels if df.columns.contains(x)]

def run_sleeper(min_s, max_s):
    """Sleeper utility to prevent server abuse"""
    sleep_time = random.randint(min_s, max_s)
    print('Sleeping for: ', sleep_time)
    time.sleep(sleep_time)

In [ ]:
# retrieval functions for different modules
functions = {
    "quote": get_equity_quote,
    "key_features": get_key_features,
    "profile": get_profile,
    "officers": get_officers
}

In [149]:
# data sourcing / hits the server
MIN_SLEEP, MAX_SLEEP = 1, 2
DATA_PATH = "./data/"

def url_open(url, symbol, module):
    print("Connecting to server", symbol, module)
    usock = urlopen(url)
    data = usock.read()
    usock.close()
    run_sleeper(MIN_SLEEP, MAX_SLEEP)    
    return data

# get JSON from server requests
def get_JSON_summary(symbol, module):
    data = url_open(JSON_URL.format(symbol, module), symbol, module)
    return json.loads(data)

def get_JSON_equity_quote(symbol):
    data = url_open(JSON_URL_QUOTE.format(symbol), symbol, module)
    return json.loads(data)

# checks whether file already exists on drive
def check_local(fname, index=None):
    found = False
    df = pd.DataFrame()
    filename = fname + '.csv'
    if os.path.isfile(DATA_PATH + filename):
        found = True
        df = pd.read_csv(DATA_PATH + filename, index_col=index, parse_dates=True)
    return found, df

# check if a symbol exists is local data store
def check_for_symbol(module, symbol):
    _, ds = check_local(module, index='symbol')
    found = False
    if (type(ds) == pd.DataFrame) and ds.index.contains(symbol):
        found = True
        print(symbol, "found on drive.")
    return found, ds

def clean_raw(symbol, result):
    df = pd.DataFrame.from_dict(result, orient='columns')
    row = pd.DataFrame(columns=df.columns)
    row.loc[symbol] = df.loc['raw']
    row = row.set_index(['symbol'], drop=True)
    row.index.name = 'symbol'
    return row

def get_equity_quote(symbol):
    module = "quote"
    flag, ds = check_for_symbol("quote", symbol)
    if flag: return ds.loc[symbol]
    converted = get_JSON_equity_quote(symbol)
    result = converted['quoteResponse']['result'][0]
    return clean_raw(symbol, result);

# retrieves key features
def get_key_features(symbol):
    module = "key_features"
    flag, ds = check_for_symbol(module, symbol)
    if flag: return ds.loc[symbol]
    params = module_map[module]
    converted = get_JSON_summary(symbol, params[0])
    result = converted['quoteSummary']['result'][0][params[0]]
    result['symbol'] = symbol
    return clean_raw(symbol, result);

# retrieves asset profile
def get_profile(symbol):
    module = "profile"
    flag, ds = check_for_symbol(module, symbol)
    if flag: return ds.loc[symbol]
    params = module_map[module]
    converted = get_JSON_summary(symbol, params[0])
    result = converted['quoteSummary']['result'][0][params[0]]
    officer_dict = result.pop('companyOfficers')
    return clean_raw(symbol, result);
    
# retrieves officers
def get_officers(symbol):
    module = "officers"
    flag, row = check_for_symbol(module, symbol)
    if flag: return ds.loc[symbol]
    params = module_map[module]
    converted = get_JSON_summary(symbol, params[0])
    result = converted['quoteSummary']['result'][0][params[0]]
    officer_dict = result.pop('companyOfficers')
    officers = pd.DataFrame()
    for o in officer_dict:
        row = pd.DataFrame.from_dict(o)
        officers = officers.append(row)
    officers = officers.loc['raw']
    officers['symbol'] = symbol
    officers.set_index('symbol', inplace=True)
    officers.index.name = 'symbol'
    return officers
    
# this function is too specific, should be used in more generic way
def get_hist_financials(symbol, stmt_key):
    flag, df = check_local(symbol + '_' + stmt_key, 'endDate')
    if flag: return df

    params = module_map[stmt_key]
    converted = get_JSON_summary(symbol, params[0])
    result = converted['quoteSummary']['result'][0][params[0]][params[1]]
    statement = pd.DataFrame()
    for period in range(len(result)):
        statement = statement.append(
            pd.DataFrame.from_dict(result[period]), sort=False)
    statement = statement.loc['raw']
    statement['endDate'] = pd.to_datetime(statement['endDate'], unit='s')
    statement.set_index('endDate', inplace=True)
    statement.to_csv(DATA_PATH + symbol + '_' + stmt_key + '.csv')
    return statement

In [152]:
# fill in functions
def fill_descriptive(model, periodEnded):
    symbol = model.index[0]
    model['date'] = date.today()
    model['periodEnded'] = periodEnded.date()
    profile_dict = get_profile(symbol).to_dict()
    model['industry'] = profile_dict['industry']
    
def fill_capital_structure(model, last4Q_bs):
    symbol = model.index[0]
    equity_quote_dict = get_equity_quote(symbol).to_dict()
    model["postMarketPrice"] = equity_quote_dict['postMarketPrice']
    model["sharesOutstanding"] = equity_quote_dict['sharesOutstanding']
    model["marketCap"] = equity_quote_dict['marketCap']
    
    cash_cols = available_labels(last4Q_bs, cash_labels)
    total_cash_df = total_fin_subset(last4Q_bs, cash_cols)
    model['totalCash'] = total_cash_df.iloc[-1]
    
    debt_cols = available_labels(last4Q_bs, debt_labels)
    total_debt_df = total_fin_subset(last4Q_bs, debt_cols)
    model['totalDebt'] = total_debt_df.iloc[-1]
    
    net_debt_df = total_debt_df - total_cash_df
    model['netDebt'] = net_debt_df.iloc[-1]
    model['minorityInterest'] = if_col_value(last4Q_bs, 'minorityInterest')
    model['enterpriseValue'] = total_fin_subset(model, tev_labels)
    
def fill_fcf(model, hist_cf):
    fcf_df = hist_cf['totalCashFromOperatingActivities'] - hist_cf['depreciation']
    model['avgNoGrowthFCF'] = fcf_df.mean()
    
def fill_intrinsic_value(model, proj_period, p1_growth, perp_growth, disc_rate):
    multiple, _, _ = simple_dcf(proj_period, p1_growth, perp_growth, disc_rate)
    equity_value = model['avgNoGrowthFCF'] * multiple - model['netDebt']
    model['projectedPeriod'] = proj_period
    model['5yearGrowth'] = p1_growth
    model['perpGrowth'] = perp_growth
    model['discountRate'] = disc_rate
    model['multiple'] = multiple
    model['equityValue'] = equity_value
    model['intrinsicValuePerShare'] = equity_value / model["sharesOutstanding"]
    model['marginOfSafety'] = 1 - model["postMarketPrice"] / model['intrinsicValuePerShare']
    

In [153]:
# modeling functions
def simple_dcf(proj_period, p1_growth, perp_growth, disc_rate):
    """Returns the unit ($1 dollar) or multiple present value of cash flows
    Args:
        proj_period: number of years out 
        p1_growth: growth rate for initial period
        perp_growth: growth into infinity
        discount_rate: discount rate for all periods
    Returns: 
        A `tuple` `(pv, projections, disc_factors)`:
            pv: The present values of future cash flows
            projections: An array with projected cash flows including terminal value
            disc_factors: An array of discount factors used
    """
    # includes the initial value + periods + terminal value
    projections = [(1 + p1_growth)**x for x in np.arange(1, proj_period + 1)]
    terminal_value = (projections[-1] *
                      (1 + perp_growth)) / (disc_rate - perp_growth)
    projections.append(terminal_value)

    disc_factors = [
        1 / ((1 + disc_rate)**x) for x in np.arange(1, proj_period + 1)
    ]
    disc_factors.append(disc_factors[-1])

    pv = np.dot(projections, disc_factors)

    return pv, projections, disc_factors

def calc_intr_value(ticker, proj_period, p1_growth, perp_growth, disc_rate):

    bv_layout_df = pd.DataFrame(index=[ticker], columns=basic_value_layout)
    
    # CF workstreams: retrieves historical and LTM cash flows
    last4A_cf = get_hist_financials(ticker, "cf")
    last4Q_cf = get_hist_financials(ticker, "cf_q")
    hist_cf = fin_append_ltm(last4A_cf, last4Q_cf)  # appends LTM to annual

    fill_descriptive(bv_layout_df, hist_cf.index[-1])
    fill_fcf(bv_layout_df, hist_cf)

    # BS workstreams: retrieves quarterly balance sheets
    last4Q_bs = get_hist_financials(ticker, "bs_q")
    fill_capital_structure(bv_layout_df, last4Q_bs)
    fill_intrinsic_value(bv_layout_df, proj_period, p1_growth, perp_growth, disc_rate)
    
    return bv_layout_df

In [5]:
# sample tickers
sample_size = 3
sample_tickers = [tickers[random.randrange(0, len(tickers))] for _ in range(sample_size)]
# on_drive = ['FB', 'AAPL', 'ANET', 'BKNG', 'LULU', 'MXIM', 'STMP', 'GMED', 'ANET', 'BKNG', 'EEFT', 'CRUS']
# sample_tickers = on_drive

In [2]:
# test parameters
ticker = "AAPL"
periods = 5
p1_growth = 0.05
PERP_GROWTH = 0.03
RISK_FREE = 0.035
RISK_PREMIUM = 0.05
disc_rate = RISK_FREE + RISK_PREMIUM

UNIT_SCALE = 10**6
nbr_fmt = lambda x: "{:.0f}".format(x / UNIT_SCALE)

### Outputs

In [ ]:
# calculates intrinsic value for a sample tickers
comp_sheet = pd.DataFrame()
for t in tickers:
    eq_val = calc_intr_value(t, periods, 0, 0, disc_rate)
    comp_sheet = comp_sheet.append(eq_val)

In [159]:
comp_sheet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, MU to ZEN
Data columns (total 20 columns):
date                      196 non-null object
periodEnded               196 non-null object
industry                  196 non-null object
postMarketPrice           196 non-null float64
sharesOutstanding         196 non-null int64
marketCap                 196 non-null int64
totalCash                 196 non-null float64
totalDebt                 196 non-null float64
netDebt                   196 non-null float64
minorityInterest          153 non-null float64
enterpriseValue           196 non-null float64
avgNoGrowthFCF            196 non-null float64
projectedPeriod           196 non-null int64
5yearGrowth               196 non-null int64
perpGrowth                196 non-null int64
discountRate              196 non-null float64
multiple                  196 non-null float64
equityValue               196 non-null float64
intrinsicValuePerShare    196 non-null float64
marginOfSafety     

In [194]:
pvt = comp_sheet.pivot_table(values='marginOfSafety', index='industry', aggfunc='mean').sort_values('marginOfSafety')
# pvt = comp_sheet.pivot_table(values='marketCap', index='industry', aggfunc='mean').sort_values('marketCap') / UNIT_SCALE
pvt.round(1) 
cols_to_show = ['marketCap','avgNoGrowthFCF','intrinsicValuePerShare','intrinsicValuePerShare','marginOfSafety']
industry = 'Software - Infrastructure'
summary = comp_sheet[comp_sheet['industry'] == industry][cols_to_show]
summary.sort_values(by='marginOfSafety')

,marketCap,avgNoGrowthFCF,intrinsicValuePerShare,intrinsicValuePerShare,marginOfSafety
NEWR,5634601984,1.051860e+07,1.113756,1.113756,-88.525900
COUP,4376421376,-3.894400e+06,4.015869,4.015869,-17.932390
MIME,2440940800,2.125340e+07,5.470107,5.470107,-6.473345
ULTI,9697922048,1.483670e+08,61.622269,61.622269,-4.037504
GWRE,7938892800,9.348500e+07,21.774044,21.774044,-3.541646
MSFT,833842184192,2.935150e+10,41.147068,41.147068,-1.640285
ORCL,189941137408,1.228700e+10,23.808875,23.808875,-1.001354
CA,18336841728,9.896000e+08,22.816814,22.816814,-0.921828
DOX,9151426560,4.812236e+08,43.992641,43.992641,-0.470701
FFIV,11566424064,6.451246e+08,148.041185,148.041185,-0.288763


### Under Construction

In [ ]:
# Goal is to pull get full JSONs and store them, parse them later

# Quote statistics
"https://query2.finance.yahoo.com/v10/finance/quoteSummary/FB?formatted=true&crumb=Feom8KtasiI&lang=en-US&region=US&modules=price%2CsummaryDetail&corsDomain=finance.yahoo.com"
# Price quote
"https://query1.finance.yahoo.com/v7/finance/quote?formatted=true&crumb=Feom8KtasiI&lang=en-US&region=US&symbols=FB&fields=messageBoardId%2ClongName%2CshortName%2CmarketCap%2CunderlyingSymbol%2CunderlyingExchangeSymbol%2CheadSymbolAsString%2CregularMarketPrice%2CregularMarketChange%2CregularMarketChangePercent%2CregularMarketVolume%2Cuuid%2CregularMarketOpen%2CfiftyTwoWeekLow%2CfiftyTwoWeekHigh&corsDomain=finance.yahoo.com"
# Historical pricing for charts, used for summary indices, commodities, 10 year bond charts
"https://query1.finance.yahoo.com/v7/finance/spark?symbols=%5EGSPC&range=1d&interval=5m&indicators=close&includeTimestamps=false&includePrePost=false&corsDomain=finance.yahoo.com&.tsrc=finance"
# Stock charts information
"https://query1.finance.yahoo.com/v8/finance/chart/FB?region=US&lang=en-US&includePrePost=false&interval=2m&range=1d&corsDomain=finance.yahoo.com&.tsrc=finance"

options_URL = 
full_summary_URL = 
quote_URL = 

data = url_open(url, symbol, module)

file = open(“testfile.txt”,”w”) 
file.close() 

In [ ]:
# UNDER CONSTRUCTION
fins_fname = "yahooFinanceAllFinancials.json"
data = ''
with open(DATA_PATH + fins_fname) as f:
    data = json.load(f)
result = data['quoteSummary']['result'][0]

bs_keys = { 'data': 'balanceSheetStatements',
           'frequency': ['balanceSheetHistory', 'balanceSheetHistoryQuarterly']}
cf_keys = { 'data': 'cashflowStatements',
           'frequency': ['cashflowStatementHistory', 'cashflowStatementHistoryQuarterly']} 
is_keys = { 'data': 'incomeStatementHistory',
           'frequency': ['incomeStatementHistory', 'incomeStatementHistoryQuarterly']} 

result[bs_keys['frequency'][1]]
# statement = pd.DataFrame()
# for period in range(len(result[bs_keys[0]])):
#     statement = statement.append(
#             pd.DataFrame.from_dict(result[period]), sort=False)
#     statement = statement.loc['raw']
#     statement['endDate'] = pd.to_datetime(statement['endDate'], unit='s')
#     statement.set_index('endDate', inplace=True)
# statement

### Unit tests

In [ ]:
# loads data sets for each module, can put it in a map by key
quote_ds = load_store_ds("quote", tickers)
keyft_ds = load_store_ds("key_features", tickers)
profile_ds = load_store_ds("profile", tickers)
officers_ds = load_store_ds("officers", tickers)

In [ ]:
last4A_cf = get_hist_financials('DBX', 'cf')
last4Q_cf = get_hist_financials('DBX', 'cf_q')
hist_cf = fin_append_ltm(last4A_cf, last4Q_cf)  # appends LTM to annual

In [177]:
# calculates intrinsic value for one ticker
# AAPL = 722532.6470588236
ivView = calc_intr_value('AAPL', periods, 0, 0, disc_rate)
print(ivView['equityValue'].loc[ticker] / UNIT_SCALE)
ivView.T

AAPL found on drive.
AAPL found on drive.
722532.6470588236


,AAPL
date,2018-09-07
periodEnded,2018-06-30
industry,Consumer Electronics
postMarketPrice,223.5
sharesOutstanding,4829929984
marketCap,1077557395456
totalCash,243743000000
totalDebt,211728000000
netDebt,-32015000000
minorityInterest,0


In [ ]:
activeCols = ['totalCash', 'totalDebt', 'netDebt', 'avgNoGrowthFCF', 'equityValue']
comp_sheet[activeCols] = (comp_sheet[activeCols] / UNIT_SCALE).round(1)
na_cols = comp_sheet.columns[comp_sheet.notnull().any()]
comp_sheet[na_cols].T

In [ ]:
stmt_key = "key_stats"
flag, key_stats_ds = check_local(stmt_key, index='symbol')
for t in sample_tickers:
    print("Getting key stats for", t)
    key_stats_ds = get_key_features(t)
key_stats_ds.to_csv(DATA_PATH + stmt_key + '.csv')

In [ ]:
# Script to check for frequency of different financial statement labels -- careful it takes a while
df = pd.DataFrame()
stmt_key = "bs_q"

for t in tickers:
    print("Retrieving info for:", t)
    # get annual balance sheets (pending impl)
    last4Q_bs = get_hist_financials(t, stmt_key) #get quarterly balance sheets
    labels = last4Q_bs.columns.tolist() #save line item labels
    dft = pd.DataFrame(columns=labels, index=[t]) # create dataframe of labels / ticker
    dft.iloc[0] = 1 # count labels for each company
    df = df.append(dft)

In [ ]:
df.to_csv(DATA_PATH + stmt_key + '_line_item_labels.csv')
# count the frequency of all labels
df.sum().sort_values()
# see if there are any random labels that occur less often, especial if they are needed for valuation

In [ ]:
last4Q_cf = get_hist_financials(ticker, "cf_q")
last4Q_cf.T.applymap(nbr_fmt) # display formatted last 4Q
last4Q_cf.sum().apply(nbr_fmt) # sums the last 4Qs

In [ ]:
""" columns set not always correct, can make dynamic by finding from 1: cf_from_ops - 1 """
cf_ops_idx = last4Q_cf.columns.tolist().index('totalCashFromOperatingActivities')
cf_ops_cols = last4Q_cf.columns[1:cf_ops_idx] 
total_fin_subset(last4Q_cf, cf_ops_cols).apply(nbr_fmt) # calculate substotal for cf from ops
total_fin_subset(ltm_cf.T, cf_ops_cols).apply(nbr_fmt)

In [ ]:
""" unit tests for simple_dcf """
pv, _, _ = simple_dcf(periods, p1_growth, PERP_GROWTH, disc_rate)
pv

In [ ]:
val_df = pd.DataFrame()
cols, rows = (10, 10)
growth = np.linspace(-6, 20, cols)
rfs = np.linspace(3, 6, rows)
ncols, nrows = np.meshgrid(growth, rfs)
""" 
ncols.round(decimals=1)
nrows.round(decimals=1)
"""

In [ ]:
# compares local and remote files for consistency
symbol = ticker
filename = symbol + '_' + stmt_key + '.csv'
stmt_key = 'cf'
filename = symbol + '_' + stmt_key + '.csv'
local = pd.read_csv(DATA_PATH + filename, index_col='endDate')
remote = get_hist_financials(symbol, stmt_key)
(local - remote).sum()

Retrives all financials in one shot.
"https://query1.finance.yahoo.com/v10/finance/quoteSummary/FB?formatted=true&crumb=Feom8KtasiI&lang=en-US&region=US&modules=incomeStatementHistory%2CcashflowStatementHistory%2CbalanceSheetHistory%2CincomeStatementHistoryQuarterly%2CcashflowStatementHistoryQuarterly%2CbalanceSheetHistoryQuarterly&corsDomain=finance.yahoo.com"

Sample:
https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com

QuoteSummary
https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?formatted=true&lang=en-US&region=US&modules=incomeStatementHistory%2CincomeStatementHistoryQuarterly&corsDomain=finance.yahoo.com

QuoteResponse
https://query1.finance.yahoo.com/v7/finance/quote?formatted=true&lang=en-US&region=US&symbols=FB&corsDomain=finance.yahoo.com